In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import glob
import re
import json
import os.path

In [2]:
paths = glob.glob("/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/ap89_collection/*")

In [3]:
data_set = []

In [4]:
x = 0q
doc_total = 0
for path in paths:
    try:
        fnames = path.split("/")[-1]
        f = open(path,"r", encoding='iso-8859-1')
        text = f.read()
        doc_total = doc_total + text.count('<DOC>')
        for m in zip(re.finditer('<DOC>', text), re.finditer('</DOC>', text)):
            docdata = text[m[0].start():m[1].end()]
            file_data = {}
            for m in zip(re.finditer('<DOCNO>', docdata), re.finditer('</DOCNO>', docdata)):
                doc_no = docdata[m[0].end() + 1 : m[1].start()]
                file_data['DOCNO'] = doc_no

            file_data['TEXT'] = ""
            for m in zip(re.finditer('<TEXT>', docdata), re.finditer('</TEXT>', docdata)):
                file_data['TEXT'] = file_data['TEXT'] + docdata[m[0].end() + 1: m[1].start()]
#             print(file_data)
            data_set.append(file_data)
    except:
        print(path + " error in file generation")

In [5]:
len(data_set)

84679

In [6]:
doc_total

84679

In [7]:
mapping = {
    "settings" : {
        "index": {
          "store": {
            "type": "fs"
          },
          "number_of_shards": 1,
          "number_of_replicas": 1
        },
        "analysis": {
          "analyzer": {
            "my_english": { 
              "type": "english",
              "stopwords_path": "stoplist.txt" 
            }
          }
        }
    },
    "mappings": {
        "doc": { 
          "properties": { 
            "DOCNO":            { "type": "text"  }, 
            "TEXT":             { 
                "type": "text",
                "store": True,
                "fielddata": True,
                "term_vector": "with_positions_offsets_payloads",
                "analyzer": "my_english"
            }, 
            "NAME":             { "type": "text"  }

          }
        }
    }
}

In [8]:
es = Elasticsearch()
es.indices.create('apdata', body=mapping)

{'acknowledged': True, 'shards_acknowledged': True}

In [9]:
index = 0
for data in data_set:
    index += 1
    res = es.index(index="apdata", doc_type='doc', id=index, body=data)

In [12]:
data_set[2997]

{'DOCNO': 'AP890630-0247 ', 'TEXT': '\n'}